## Základný opis dát spolu s ich charakteristikami ##

### Adrian Maslak ###

Import kniznic

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.spatial import cKDTree

Nacitanie datasetu

In [ ]:
patient = pd.read_csv('../069/patient.csv', sep="\t", quotechar='"', engine='python')
station = pd.read_csv('../069/station.csv' , sep="\t")
observation = pd.read_csv('../069/observation.csv', sep="\t")

Basic information

In [ ]:
import pandas as pd



# Over typy dát pre latitude a longitude
observation['latitude'] = pd.to_numeric(observation['latitude'], errors='coerce')
observation['longitude'] = pd.to_numeric(observation['longitude'], errors='coerce')
station['latitude'] = pd.to_numeric(station['latitude'], errors='coerce')
station['longitude'] = pd.to_numeric(station['longitude'], errors='coerce')

# Pridaj station_ID
station['station_ID'] = station.index + 1

# Spoj observation so station na latitude a longitude
merged_obs_station = pd.merge(observation, station, on=['latitude', 'longitude'], how='left')

# Over, či station_ID je numerický
merged_obs_station['station_ID'] = pd.to_numeric(merged_obs_station['station_ID'], errors='coerce')
patient['station_ID'] = pd.to_numeric(patient['station_ID'], errors='coerce')

# Spoj s patient na station_ID
merged_all = pd.merge(merged_obs_station, patient, on='station_ID', how='left')

# Zobraz štruktúru výsledného datasetu
print("Počet riadkov a stĺpcov v spojenom datasete:", merged_all.shape)
print("\nAtribúty a ich typy:\n", merged_all.dtypes)

# Ulož výsledok
merged_all.to_csv('merged_dataset.csv', index=False)

In [ ]:

# Funkcia na analýzu štruktúry datasetu
def analyze_structure(df, name):
    print(f"--- Analýza štruktúry pre {name.upper()} ---")
    print(f"Počet záznamov: {df.shape[0]}")
    print(f"Počet atribútov: {df.shape[1]}")
    print("Typy atribútov:")
    display(df.dtypes)
    print("Chýbajúce hodnoty (počet a %):")
    null_counts = df.isnull().sum()
    null_percent = (null_counts / df.shape[0] * 100).round(2)
    display(pd.concat([null_counts, null_percent], axis=1, keys=['Počet null', '% null']))
    
    # Unikátne hodnoty
    print("Unikátne hodnoty v kľúčových stĺpcoch:")
    key_columns = ['user_id', 'station_ID', 'blood_group', 'company'] if name == 'patient' else \
                  ['location', 'station', 'code'] if name == 'station' else []
    for col in key_columns:
        if col in df.columns:
            print(f"{col}: {df[col].nunique()}")
            print(f"Top 5 hodnôt v {col}:")
            display(df[col].value_counts().head())
    
    # Duplikáty
    print(f"Počet duplicitných riadkov: {df.duplicated().sum()}")
    if 'user_id' in df.columns:
        print(f"Počet duplicitných user_id: {df['user_id'].duplicated().sum()}")
        if df['user_id'].duplicated().sum() > 0:
            print("Príklady duplicitných user_id (prvé 5):")
            display(df[df['user_id'].duplicated(keep=False)].sort_values('user_id').head())
    

# Aplikácia na všetky datasety
for df, name in [(patient, 'patient'), (station, 'station'), (observation, 'observation')]:
    analyze_structure(df, name)



# A

## Analyza dat: <br>
Dataset *patient* obsahuje 2064 zaznamov, 15 atributov
__Chybajuce Hodnoty __
- residence: 100%
- job: 70% 
- adress: 15%
- location: 5%

Obsahuje mnoho duplicitnych __user_id__, pomerne chaoticky. Neskor zistime ci je potrebny pre nas ciel
<br>

Dataset *station* obsahuje 836 zaznamov, 6 atributov
- Tento dataset je pomerne upratany, bez duplikatov 

Dataset *observation* obsahuje 12042 zaznamov, 23 atributov
- Nas hlavny dataset pre predikciu _oximetry_. Je bez duplikatov a chybajucich hodnot. Vsetky hodnoty su typu float64





# B   
## Analyza jednotlivych udajov 


In [ ]:

selected_attrs = ['SpO₂', 'HR', 'PI', 'RR', 'EtCO₂', 'FiO₂', 'BP', 'Skin Temperature', 'Hb level', 'oximetry']

# Očakávané medicínske rozsahy
ranges = {
    'SpO₂': (95, 100),
    'HR': (60, 100),
    'PI': (0.2, 20),
    'RR': (12, 20),
    'EtCO₂': (35, 45),
    'FiO₂': (21, 100),
    'BP': (90, 120),
    'Skin Temperature': (33, 38),
    'Hb level': (12, 18),
    'oximetry': (0, 1)
}

# Deskriptívne štatistiky pre observation
print("Deskriptívne štatistiky pre vybrané atribúty v observation:")
display(observation[selected_attrs].describe())

# Overenie rozsahov
print("\nOverenie medicínskych rozsahov:")
for attr in selected_attrs:
    min_val, max_val = observation[attr].min(), observation[attr].max()
    expected_min, expected_max = ranges.get(attr, (None, None))
    status = 'OK' if (expected_min <= min_val <= max_val <= expected_max) else 'Mimo rozsah'
    print(f"{attr}: Min={min_val:.2f}, Max={max_val:.2f}, Očakávaný rozsah={expected_min}-{expected_max}, Status={status}")

# Vizualizácia distribúcií (histogramy s KDE)
fig, axes = plt.subplots(5, 2, figsize=(12, 20))
axes = axes.ravel()
for i, attr in enumerate(selected_attrs):
    sns.histplot(observation[attr], kde=True, ax=axes[i])
    axes[i].set_title(f"Distribúcia {attr}")
plt.tight_layout()
plt.show()

# Boxploty pre numerické atribúty
fig, axes = plt.subplots(5, 2, figsize=(12, 20))
axes = axes.ravel()
for i, attr in enumerate(selected_attrs):
    sns.boxplot(y=observation[attr], ax=axes[i])
    axes[i].set_title(f"Boxplot {attr}")
plt.tight_layout()
plt.show()

# Analýza blood_group z patient
print("\nDistribúcia blood_group v patient:")
display(patient['blood_group'].value_counts(normalize=True).round(3))
plt.figure(figsize=(8, 6))
sns.countplot(x='blood_group', data=patient, order=patient['blood_group'].value_counts().index)
plt.title("Distribúcia blood_group")
plt.xticks(rotation=45)
plt.show()

V deskriptivnej statistike vidime: 
- $SpO_2$ (Saturacia kyslika): Hodnoty vyzeraju normalne. Nemame ziadne hodnoty pod 95 co znaci pozitivne vysledky.
- HR (Srdcovy tep): Priemer je ~80 BPM. Rozsah 60-100 BPM co znaci normalnu distribuciu. Nevidime ziadne extremy
- PI (Perfuzny index): Priemer ~10, znaci zdravu populaciu. Histogram je mierne Left-skewed co znaci ze sa nova krv dostava do koncatim vo vacsom mnostve vo vacsej casti populacie. Rozsah 0.2-20 - normalne hodnoty su 1-20
- RR (dychacia frekvencia): Priemer ~16, normalny rozsah je 12-20 co znaci zdrave hodnoty. Nevidime ziadne extremne hodnoty co by znacilo dychavicnost alebo ine priznaky
- $EtCO_2$ (end-tidal CO2) : priemer 39.81mmHg co je v sulade s priemerom
- $FiO_2$ (Frakcia inspirovaneho kyslika): Priemer ~64% - vyssi priemer naznacuje ze cast pacientov je na kyslikovej liecbe. Hladina kyslika vo vzduchu 0.21, extremne pripady pri terapii 1.00. 
- BP (Systolicky krvny tlak) : priemer ~103 mmHg - normalne hodnoty su 90-120 cize hodnoty su v norme. 
- Skin temperature: Priemer ~35,5 C z mojho hladiska je teplota mierne pod beznou telesnou teplotou. No v pripade zistovanych lekarskych hodnot je normalny rozsah 33-37 C cize tiez v norme
- HB (hemoglobin): Priemer ~15, rozsah je 12-18 co neznaci ziadne znamky anemie alebo polycytemie
- oximetry: Nechapem (Potrebujem konzultovat)


<br>
Krvne skupiny su rovnomerne rozdelene co nam vytvara vhodny kategoricky model 

# C
## Parova analyza

In [ ]:
corr_matrix = observation.corr(method='pearson')

# veľkosť a ukážka matice
print(f"Rozmery korelačnej matice: {corr_matrix.shape}")
corr_matrix.head()

# Vizualizácia heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", vmin=-1, vmax=1)
plt.title("Korelačná matica atribútov")
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# dvojice atribútov, ktoré chceme zobraziť
pairs_to_plot = [
    ('HR', 'CO'),
    ('SpO₂', 'oximetry'),
    ('SpO₂', 'PI'),
    ('Skin Temperature', 'PI'),
    ('Skin Temperature', 'RR')
]
asd
# jednoduché scatterploty
for x, y in pairs_to_plot:
    if x in observation.columns and y in observation.columns:
        plt.figure(figsize=(6,5))
        sns.scatterplot(data=observation, x=x, y=y, alpha=0.6)
        plt.title(f'{x} vs {y}', fontsize=13)
        plt.xlabel(x)
        plt.ylabel(y)
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()
        plt.show()


In [ ]:
data['oximetry'].value_counts(normalize=True)
